<a href="https://colab.research.google.com/github/AgPriyank/react_agent/blob/main/react_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/AgPriyank/react_agent

In [ ]:
!pip install openai

In [ ]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
from openai import OpenAI
client = OpenAI()

In [ ]:
import os
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
responses=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role":"user","content":"Say Hello"}]
)

print(responses.choices[0].message.content)


In [ ]:
client =OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
responses=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role":"user","content":"What is 2+2?"}]
)
print(responses.choices[0].message.content)

In [ ]:
def askllm(question):
  responses=client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{"role":"user","content":question}]
  )
  return responses.choices[0].message.content

In [ ]:
askllm("What is the capital of India?")

In [ ]:
### text parsing text contains Action and Action input

def parse_text(text):
  text=text.lower()
  text=text.replace("\t"," ")
  text=text.strip()
  lines=text.split("\n")
  action_acquired=0
  action_input_acquired=0
  thought_acquired=0
  for line in lines:
    line_strip=line.split(":")
    if len(line_strip)!=2:
      continue
    key_to_compare = line_strip[0]
    value_to_assign = line_strip[1]
    key_to_compare=key_to_compare.strip()
    value_to_assign=value_to_assign.strip()
    if "action" in key_to_compare:
      if "action input" in key_to_compare and action_input_acquired == 0:
        action_input=value_to_assign
        action_input_acquired = 1
      elif action_acquired==0:
        action=value_to_assign
        action_acquired=1
    elif "thought" in key_to_compare and thought_acquired == 0:
      thought=value_to_assign
      thought_acquired=1
  return action,action_input, thought

In [ ]:
text= """
   Thought:    I need to search
   Action:     search
   Action Input:     Paris population
"""
parse_text(text)

In [ ]:
text2 = """
Thought: I should search for the population
Action: search
Action Input: Paris population
"""

text1 = """
Thought: I need to calculate
Action: calculator
Action Input: 2 + 2
"""

text4 = """
Thought: I need to compute the percentage
Action: calculator
Action Input: 150 * 0.20 """

In [ ]:
parse_text(text4)

In [ ]:
!pip install asteval

In [ ]:
from asteval import Interpreter
aeval = Interpreter()
import re
def calculator(expression):
    """
    Takes a math expression as string, returns result.
    Example: calculator("2 + 2") -> 4
    """
    expression=re.sub(r"\s*of\s*","*",expression)
    expression= re.sub(r'(\d+)%', r'\1/100', expression)
    try:
      answer = aeval(expression)
      return str(answer)
    except:
      return "Invalid expression"


In [ ]:
calculator("10% of 200")

In [ ]:
def search(query):
    """
    Pretend to search, return fake results.
    """
    fake_db = {
        "paris": "Paris is the capital of France with a population of 2.2 million",
        "france": "France is a country in Europe. Its capital is Paris",
        "capital": "The capital of France is Paris"
    }

    query_lower= query.lower()

    for key in fake_db.keys():
      if key in query_lower:
        return fake_db[key]
    else:
      return "not found"



In [ ]:
question="What is the population of the capital of France?"
prompt=f"""Answer in the following format

Thought: think about what to do
Action: one of [search, calculator, finish]
Action input: input for the action


IMPORTANT: Output only ONE action at a time, then STOP. Wait for the Observation before continuing.

Available actions:
- search: look up information
- calculator: evaluate a math expression
- finish: give the final answer

Question: {question}"""

import os
from openai import OpenAI

client=OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


n=3
for i in range(n):
  print(f"interation\n---{i+1}----\n")

  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{"role":"user","content":prompt}],
      stop=["Observation"],
      temperature=0
  )
  output=response.choices[0].message.content
  print(f"complete output\n--{output}\n")

  action,action_input, thought = parse_text(output)

  if action == "finish":
    print(f"final answer : {action_input}\n")
    break
  elif action == "calculator":
    observation=calculator(action_input)
  elif action =="search":
    observation=search(action_input)
  else:
    print(f"invalid action {action}\n")

  print(f"observation {observation}\n")

  prompt = prompt + f"""\n{output}\n Observation: {observation}\n If you think you are done then reply with
  Thought: I am done
  Action: Finish
  Action Input: Give the last observation as the final answer"""




In [ ]:
"action input" in "action input "

In [ ]:
import os
from openai import OpenAI
from operator import attrgetter
class Tool:
  def __init__(self,name, func, description):
    self.name=name
    self.func=func
    self.description = description

  def apply_tool(self, input_text):
    return self.func(input_text)





class ReActAgent:
  def __init__(self, tools, max_iterations=5):
    self.tools = {tool.name: tool for tool in tools}
    self.max_iterations=max_iterations


  def format_prompt(self, question):

    for tool in self.tools:
      print(tool)
    tool_names = [self.tools[tool].name for tool in self.tools]
    tool_description = [self.tools[tool].description for tool in self.tools]
    print(tool_names)
    print(tool_description)

    prompt= f"""Answer in the following format

    Thought: think about what to do
    Action: one of {tool_names}
    Action input: input for the action


    IMPORTANT: Output only ONE action at a time, then STOP. Wait for the Observation before continuing.

    Available actions are {tool_names} with description {tool_description}
    - search: look up information
    - calculator: evaluate a math expression
    - finish: give the final answer

    Question: {question}"""

    return prompt


  def parse_text(self,text):
    text=text.lower()
    text=text.replace("\t"," ")
    text=text.strip()
    lines=text.split("\n")
    action_acquired=0
    action_input_acquired=0
    thought_acquired=0
    for line in lines:
      line_strip=line.split(":")
      if len(line_strip)!=2:
        continue
      key_to_compare = line_strip[0]
      value_to_assign = line_strip[1]
      key_to_compare=key_to_compare.strip()
      value_to_assign=value_to_assign.strip()
      if "action" in key_to_compare:
        if "action input" in key_to_compare and action_input_acquired == 0:
          action_input=value_to_assign
          action_input_acquired = 1
        elif action_acquired==0:
          action=value_to_assign
          action_acquired=1
      elif "thought" in key_to_compare and thought_acquired == 0:
        thought=value_to_assign
        thought_acquired=1
    return action,action_input, thought



  def run(self,question):

    client=OpenAI(api_key=os.environ.get("OPENAPI_GET_KEY"))

    init_prompt = self.format_prompt(question)

    for i in range(self.max_iterations):
      print(f"Iteration index : {i+1}\n")

      response= client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[{"role":"user","content":init_prompt}],
          temperature=0,
          stop=["Observation"]
      )

      output=response.choices[0].message.content

      print(output)

      action,action_input, thought=self.parse_text(output)


      observation=action_input
      if action == "finish":
        print(f"final answer : {action_input}\n")
        break
      else:
        tool_to_use = self.tools[action]
        observation = tool_to_use.apply_tool(action_input)
        print(f"observation {observation}\n")

      init_prompt = init_prompt + f"""\n{output}\n Observation: {observation}\n If you think you are done then reply with
      Thought: I am done
      Action: Finish
      Action Input: Give the last observation as the final answer"""











In [ ]:
calculator_tool = Tool(
    name="calculator",
    func=calculator,
    description="Calculates math expressions"
)

search_tool = Tool(
    name="search",
    func=search,
    description="Searches for information"
)

In [ ]:
agent = ReActAgent([calculator_tool, search_tool])

In [ ]:
answer = agent.run("What is 20% of 500?")

In [ ]:
search_tool